In [7]:
%pylab nbagg
from operatorsmatrix import *
from projections import build_projection_operator
import numpy as np
from utils import ims
import scipy.sparse
import scipy.sparse.linalg

Populating the interactive namespace from numpy and matplotlib


/scisoft/users/paleo/virtualenvs/cuttingedge/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['pi', 'sqrt', 'gradient']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
def coherence_identity(A):
    # needs a recent version of scipy to use sparse.linalg.norm()
    # eg. for 256x256 images, angles from -70 to 70 with step 5 = 29 angles
    # A = build_projection_operator(256, 29)
    # should give a coherence of 30.3 (max is 256), compatible with Guay Et Al
    norm_A = scipy.sparse.linalg.norm(A, axis=1)
    A = (A.T / norm_A).T
    coherence = A.max() * sqrt(A.shape[1]) 
    return coherence


In [17]:
def coherence_normalized(A, B):
    N = A.shape[1]
    A2 = np.linalg.norm(A, axis=1)
    B2 = np.linalg.norm(B, axis=1)
    # In case where a line of A or B is zero (happens only for gradient)
    A2[A2 == 0] = 1
    B2[B2 == 0] = 1
    return np.max(np.abs(A.dot(B.T)/np.outer(A2,B2)))*sqrt(N)   # normalize by the product of the norms of the basis functions


def coherence_normalized_sparse(A, B):
    # works for sparse (csr) matrices A, B
    N = A.shape[1]
    norm_A = scipy.sparse.linalg.norm(A, axis=1) # np.ndarray
    norm_B = scipy.sparse.linalg.norm(B, axis=1) # np.ndarray
    # In case where a line of A or B is zero (happens only for gradient)
    norm_A[norm_A == 0] = 1
    norm_B[norm_B == 0] = 1
    dotprods = A.dot(B.T) # sparse.csr_matrix
    # normalize
    for i in range(A.shape[0]):
        dotprods[i, :] /= norm_A[i]
    for j in range(N):
        dotprods[:, i] /= norm_B[j]
    # take abs val
    dotprods = dotprods.maximum(0) # TODO: true abs val !
    # return max * sqrt(N)
    return dotprods.max()*sqrt(N)


def coherence_onthefly(A, B):
    




In [27]:
nbins = 128
Npj = [32, 64, 80]


In [28]:
coh_identity = []
coh_haar = []
coh_coif = []
coh_db2 = []
coh_grad = []
for npj in Npj:
    print("(%d, %d)" % (nbins, npj))
    A = build_projection_operator(nbins, n_dir=npj)
    coh_identity.append(coherence_identity(A))
    #
    P = np.array(A.todense())
    #
    W = build_wavelet_matrix_2d(nbins, wname="haar")
    coh_haar.append(coherence_normalized(P, W))
    #
    W = build_wavelet_matrix_2d(nbins, wname="coif1")
    coh_coif.append(coherence_normalized(P, W))
    #
    W = build_wavelet_matrix_2d(nbins, wname="db4")
    coh_db2.append(coherence_normalized(P, W))
    #
    G = build_gradient_matrix_2d(nbins, merged=True)
    coh_grad.append(coherence_normalized(P, G))
    
    

        

(128, 32)
(128, 64)
(128, 80)


In [29]:
print coh_identity
print coh_haar
print coh_coif
print coh_db2 
print coh_grad 

[21.265262603759766, 21.266286849975586, 21.286754608154297]
[25.372442287367768, 25.372442287367768, 25.562762669794441]
[33.066762696856777, 34.197069428787131, 34.379565565484228]
[29.452493896385743, 29.594822989789737, 30.218495056675732]
[13.844735186817632, 13.844735186817632, 13.878481114042406]


The coherence does not depend on the number of projections.

In [45]:
Nbins = [32, 64, 128, 256]
Nproj = [32, 64, 80, 80] # corresponding nproj

In [46]:
coh_identity = []
coh_haar = []
coh_coif = []
coh_db2 = []
coh_grad = []
for nbins, npj in zip(Nbins, Nproj):
    print("(%d, %d)" % (nbins, npj))
    #
    """
    A = build_projection_operator(nbins, n_dir=npj)
    coh_identity.append(coherence_identity(A))
    #
    P = np.array(A.todense())
    #
    W = build_wavelet_matrix_2d(nbins, wname="haar")
    coh_haar.append(coherence_normalized(P, W))
    #
    W = build_wavelet_matrix_2d(nbins, wname="coif1")
    coh_coif.append(coherence_normalized(P, W))
    #
    W = build_wavelet_matrix_2d(nbins, wname="db2")
    coh_db2.append(coherence_normalized(P, W))
    #
    """
    G = build_gradient_matrix_2d(nbins, merged=True)
    coh_grad.append(coherence_normalized(P, G))
    
    
print coh_grad 

(32, 32)


Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


ValueError: shapes (4096,4096) and (1024,2048) not aligned: 4096 (dim 1) != 1024 (dim 0)

In [22]:
print coh_identity
print coh_haar
print coh_coif
print coh_db2 
print coh_grad 

[10.124059677124023, 14.84012508392334, 21.286754608154297, 30.255407333374023]
[10.939597852847299, 16.12101026296412, 25.562762669794441, 39.299696080414222]
[13.549781087961017, 22.015971327323005, 34.379565565484228, 50.343450115412324]
[12.557505732598489, 20.374810066087932, 30.218495056675732, 44.682133693914068]
[6.5682807507974852, 9.6245014934864805, 13.878481114042406]
